In [1]:
import pandas as pd
from obd import bts_parquet_path, bts_dat_path, bts_all_path, bts_csv_path, preprocess, downcast, y_name, cost_name, prob_name
from cb import df_to_dat, df_to_vw_format
from utils import iter_partitions, iter_parquets
from os.path import join
from sklearn.preprocessing import OrdinalEncoder
from vowpalwabbit import Workspace
from pandas.api.types import is_integer_dtype
import numpy as np

In [ ]:
df = pd.read_parquet(bts_parquet_path)

In [ ]:
for col, v in df.nunique().items():
    if v > 100:
        print(col)

In [ ]:
df, x_encoder = preprocess(df)
df = downcast(df)
df.head()

In [ ]:
converted = df_to_vw_format(df, y_name, cost_name, prob_name)
converted.to_csv("converted.csv")

In [2]:
converted = pd.read_csv("converted.csv")["0"]

In [3]:
model = Workspace("--cb 80", enable_logging=True)

In [11]:
for i, v in converted.items():
    model.learn(v)